## Data Loading
Blah

In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing

train = pd.DataFrame.from_csv('../../train.csv')
test = pd.DataFrame.from_csv('../../test.csv')

## Dimensions of train set
ntrain,dtrain = train.shape

## Dimensions of test set
ntest, dtest = test.shape

## Standardize data

In [2]:
X = train.drop(['TARGET'], axis=1)
Xtest = test

## Standardize sets together
Xtotal = X.append(Xtest)
Xtotal_scaled = preprocessing.scale(Xtotal)
X_scaled,Xtest_scaled = np.split(Xtotal_scaled, [ntrain])

targets = np.array(train.TARGET)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


## Make Validation Set

In [3]:
from sklearn import cross_validation

sss = cross_validation.StratifiedShuffleSplit(targets, 1, test_size=0.2, random_state=0)
(train_index,valid_index) = list(sss)[0]

X_train = X_scaled[train_index,]
X_valid = X_scaled[valid_index,]
y_train = targets[train_index]
y_valid = targets[valid_index]

print(X_train.shape)
print(X_valid.shape)

(60816, 369)
(15204, 369)


## Run the thing (Linear)

In [7]:
from sklearn import svm
from sklearn.metrics import roc_auc_score

lin_clf = svm.LinearSVC(dual=False, class_weight="balanced")
lin_clf.fit(X_train, y_train)

y_hat = lin_clf.predict(X_valid)
y_hat2 = lin_clf.predict(X_train)
print("Training accuracy:",lin_clf.score(X_train, y_train))
print("Validation accuracy:",lin_clf.score(X_valid, y_valid))
print("Training AUC:",roc_auc_score(y_train, y_hat2))
print("Validation AUC:",roc_auc_score(y_valid, y_hat))

('Training accuracy:', 0.69721783741120757)
('Validation accuracy:', 0.69258089976322024)
('Training AUC:', 0.72561025957619218)
('Validation AUC:', 0.70139324654475499)


## Run the thing (Polynomial kernel)
This takes almost forever to run

In [8]:
from sklearn import svm
from sklearn.metrics import roc_auc_score

clfp = svm.SVC(kernel="poly", degree=3, class_weight="balanced")
clfp.fit(X_train, y_train)

y_hat = clfp.predict(X_valid)
y_hat2 = clfp.predict(X_train)
print("Training accuracy:",clfp.score(X_train, y_train))
print("Validation accuracy:",clfp.score(X_valid, y_valid))
print("Training AUC:",roc_auc_score(y_train, y_hat2))
print("Validation AUC:",roc_auc_score(y_valid, y_hat))

('Training accuracy:', 0.75419297553275455)
('Validation accuracy:', 0.74026571954748754)
('Training AUC:', 0.72398845806217216)
('Validation AUC:', 0.66330910388191489)


## Run the thing (RBF kernel)
This also takes almost forever to run

In [10]:
from sklearn import svm
from sklearn.metrics import roc_auc_score

clfp = svm.SVC(kernel="rbf", class_weight="balanced")
clfp.fit(X_train, y_train)

y_hat = clfp.predict(X_valid)
y_hat2 = clfp.predict(X_train)
print("Training accuracy:",clfp.score(X_train, y_train))
print("Validation accuracy:",clfp.score(X_valid, y_valid))
print("Training AUC:",roc_auc_score(y_train, y_hat2))
print("Validation AUC:",roc_auc_score(y_valid, y_hat))

('Training accuracy:', 0.70057221783741119)
('Validation accuracy:', 0.69060773480662985)
('Training AUC:', 0.75684607177485153)
('Validation AUC:', 0.68205056331881908)
